In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta

# Load Data
options_df = pd.read_csv(r"D:\CODES\Banknifty Data\Banknifty_Options_2018-July_2023.csv")
spot_df = pd.read_csv(r"D:\CODES\Banknifty Data\Banknifty_Spot_2018-2023.csv")

# Convert 'Date' columns to datetime format
options_df['Date'] = pd.to_datetime(options_df['Date'])
spot_df['Date'] = pd.to_datetime(spot_df['Date'])

# Convert 'ExpiryDate' to datetime format
options_df['ExpiryDate'] = pd.to_datetime(options_df['ExpiryDate'], format='%d%b%y')

# Ensure the 'Time' column is in a correct format if it exists
if 'Time' in options_df.columns:
    options_df['Time'] = options_df['Time'].str.strip()  # Remove leading/trailing spaces
    options_df['Time'] = pd.to_datetime(options_df['Time'], format='%H:%M:%S').dt.time
    options_df['Datetime'] = options_df.apply(lambda row: pd.Timestamp.combine(row['Date'], row['Time']), axis=1)

if 'Time' in spot_df.columns:
    spot_df['Time'] = spot_df['Time'].str.strip()  # Remove leading/trailing spaces
    spot_df['Time'] = pd.to_datetime(spot_df['Time'], format='%H:%M:%S').dt.time
    spot_df['Datetime'] = spot_df.apply(lambda row: pd.Timestamp.combine(row['Date'], row['Time']), axis=1)

# Filter for January 2018
options_df_2018 = options_df[(options_df['Date'].dt.year == 2018) & (options_df['Date'].dt.month == 1)].copy()
spot_df_2018 = spot_df[(spot_df['Date'].dt.year == 2018) & (spot_df['Date'].dt.month == 1)].copy()

# Delete original DataFrames to free up memory
del options_df
del spot_df

# Further filter options_df_2018 for "PE" instruments
pe_options_df_2018 = options_df_2018[options_df_2018["Instrument Type"] == "PE"].copy()

# Select only necessary columns to reduce memory usage
necessary_columns = ['Date', 'Ticker', 'Strike Price', 'ExpiryDate', 'Time', 'Close', 'Datetime']
pe_options_df_2018 = pe_options_df_2018[necessary_columns]
spot_df_2018 = spot_df_2018[['Date', 'Time', 'Close', 'Datetime']]

# Initialize global trade log DataFrame
global trade_log_df
trade_log_df = pd.DataFrame()

def calculate_atm_strike_price_for_date(spot_df, selected_date, entry_time):
    trade_time = pd.to_datetime(entry_time, format='%H:%M:%S').time()
    filtered_df = spot_df[(spot_df['Date'] == pd.to_datetime(selected_date)) & (spot_df['Time'] == trade_time)]
    if not filtered_df.empty:
        avg_close_price = filtered_df['Close'].mean()
        atm_strike_price = round(avg_close_price / 100) * 100
        return atm_strike_price
    else:
        return None

def put_log_trades(trade_log, selected_date, closest_expiry, entry_time, atm_strike_price, put_options_atm, put_options_otm, put_options_itm, capital, stop_loss, total_premium, target, max_profit, max_loss):
    trades = {
        'Serial_No': len(trade_log_df) + 1,
        'Date': selected_date,
        'Expiry': closest_expiry,
        'En_Date': selected_date,
        'Pe_En_Date': selected_date,
        'Ce_Ex_Date': None,
        'Pe_Ex_Date': None,
        'Atm_Strike': atm_strike_price,
        'Pe_Short_Strike': atm_strike_price,
        'Pe_otm_Long_Strike': put_options_otm['Strike Price'],
        'Pe_itm_Long_Strike': put_options_itm['Strike Price'],
        'Capital': capital,
        'Pe_Short_En_Price': put_options_atm['Close'],
        'Pe_Short_Ex_Price': None,
        'Pe_otm_Long_En_Price': put_options_otm['Close'],
        'Pe_otm_Long_Ex_Price': None,
        'Pe_itm_Long_En_Price': put_options_itm['Close'],
        'Pe_itm_Long_Ex_Price': None,
        'Entry_Time': entry_time,
        'Exit Time': None,
        'Current Market Price': None,
        'Ticker_Pe_Short': 'BANKNIFTY',
        'Ticker_otm_Pe_Long': 'BANKNIFTY',
        'Ticker_itm_Pe_Long': 'BANKNIFTY',
        'Exit_Reason': None,
        'Stop_Loss': stop_loss,
        'Total_Premium_Received': total_premium,
        'Target': target,
        'Max_Profit': max_profit,
        'Max_Loss': max_loss,
        'Total_PnL': None
    }
    trade_log.append(trades)
    return trade_log

def get_option_market_price(options_df, current_datetime, strike_price, expiry_date):
    df = options_df[(options_df['Datetime'] == current_datetime) & 
                    (options_df['Strike Price'] == strike_price) & 
                    (options_df['ExpiryDate'] == expiry_date)]
    if not df.empty:
        current_price = df.iloc[-1]['Close']  # Get the most recent closing price
        return current_price
    else:
        return None

def calculate_pnl(trade_log_df, options_df, current_datetime):
    atm_short_current_price = get_option_market_price(options_df, current_datetime, trade_log_df['Pe_Short_Strike'].values[0], trade_log_df['Expiry'].values[0])
    otm_long_current_price = get_option_market_price(options_df, current_datetime, trade_log_df['Pe_otm_Long_Strike'].values[0], trade_log_df['Expiry'].values[0])
    itm_long_current_price = get_option_market_price(options_df, current_datetime, trade_log_df['Pe_itm_Long_Strike'].values[0], trade_log_df['Expiry'].values[0])

    if None in [atm_short_current_price, otm_long_current_price, itm_long_current_price]:
        return None  # If any prices are not found, return None

    pe_short_pnl = trade_log_df['Pe_Short_En_Price'].values[0] - atm_short_current_price
    pe_otm_long_pnl = otm_long_current_price - trade_log_df['Pe_otm_Long_En_Price'].values[0]
    pe_itm_long_pnl = itm_long_current_price - trade_log_df['Pe_itm_Long_En_Price'].values[0]

    total_pnl = pe_short_pnl + pe_otm_long_pnl + pe_itm_long_pnl
    trade_log_df['Total_PnL'] = total_pnl
    return total_pnl

def monitor_trades(trade_log_df, options_df, start_datetime, end_time=time(15, 15, 59)):
    current_time = start_datetime
    row_number = trade_log_df.index[-1]  # Reference the latest trade entry
    
    while current_time.time() <= end_time:
        pnl = calculate_pnl(trade_log_df, options_df, current_time)
        if pnl is None:
            print("Market data not available for", current_time)
        else:
            print("Current PnL:", pnl)
            
            # Add pnl to trade_log_df 
            trade_log_df.at[row_number, 'Total_PnL'] = pnl
            
            if pnl <= trade_log_df.at[row_number, 'Stop_Loss']:
                print("Trade stopped due to stop loss hit.")
                trade_log_df.at[row_number, 'Exit Time'] = current_time
                trade_log_df.at[row_number, 'Pe_Short_Ex_Price'] = get_option_market_price(options_df, current_time, trade_log_df.at[row_number, 'Pe_Short_Strike'], trade_log_df.at[row_number, 'Expiry'])
                trade_log_df.at[row_number, 'Pe_otm_Long_Ex_Price'] = get_option_market_price(options_df, current_time, trade_log_df.at[row_number, 'Pe_otm_Long_Strike'], trade_log_df.at[row_number, 'Expiry'])
                trade_log_df.at[row_number, 'Pe_itm_Long_Ex_Price'] = get_option_market_price(options_df, current_time, trade_log_df.at[row_number, 'Pe_itm_Long_Strike'], trade_log_df.at[row_number, 'Expiry'])
                break
            elif pnl >= trade_log_df.at[row_number, 'Target']:
                print("Trade stopped due to target hit.")
                trade_log_df.at[row_number, 'Exit Time'] = current_time
                trade_log_df.at[row_number, 'Pe_Short_Ex_Price'] = get_option_market_price(options_df, current_time, trade_log_df.at[row_number, 'Pe_Short_Strike'], trade_log_df.at[row_number, 'Expiry'])
                trade_log_df.at[row_number, 'Pe_otm_Long_Ex_Price'] = get_option_market_price(options_df, current_time, trade_log_df.at[row_number, 'Pe_otm_Long_Strike'], trade_log_df.at[row_number, 'Expiry'])
                trade_log_df.at[row_number, 'Pe_itm_Long_Ex_Price'] = get_option_market_price(options_df, current_time, trade_log_df.at[row_number, 'Pe_itm_Long_Strike'], trade_log_df.at[row_number, 'Expiry'])
                break
        current_time += timedelta(minutes=1)  # Increment time by 1 minute

    if 'Exit Time' not in trade_log_df.columns or pd.isnull(trade_log_df.at[row_number, 'Exit Time']):
        trade_log_df.at[row_number, 'Exit Time'] = end_time
        trade_log_df.at[row_number, 'Pe_Short_Ex_Price'] = get_option_market_price(options_df, end_time, trade_log_df.at[row_number, 'Pe_Short_Strike'], trade_log_df.at[row_number, 'Expiry'])
        trade_log_df.at[row_number, 'Pe_otm_Long_Ex_Price'] = get_option_market_price(options_df, end_time, trade_log_df.at[row_number, 'Pe_otm_Long_Strike'], trade_log_df.at[row_number, 'Expiry'])
        trade_log_df.at[row_number, 'Pe_itm_Long_Ex_Price'] = get_option_market_price(options_df, end_time, trade_log_df.at[row_number, 'Pe_itm_Long_Strike'], trade_log_df.at[row_number, 'Expiry'])
    print("Monitoring ended.")

def create_straddle(pe_options_df, spot_df, selected_date, entry_time, capital):
    global trade_log_df

    # Calculate the ATM strike price for the selected date
    atm_strike_price = calculate_atm_strike_price_for_date(spot_df, selected_date, entry_time)
    if atm_strike_price is None:
        print(f"No ATM strike price found for {selected_date} at {entry_time}.")
        return pd.DataFrame()

    # Filter options data for the specified date and entry time
    options_for_date = pe_options_df[pe_options_df['Datetime'] == pd.to_datetime(f"{selected_date} {entry_time}")]

    if options_for_date.empty:
        print(f"No options data found for {selected_date} at {entry_time}.")
        return pd.DataFrame()

    # Find the closest expiry date to the selected date
    closest_expiry = options_for_date['ExpiryDate'].min()

    # Filter options data for the closest expiry date
    options_for_closest_expiry = options_for_date[options_for_date['ExpiryDate'] == closest_expiry]

    # Calculate the 2% away strikes and round them to the nearest 100 points
    strike_increment = atm_strike_price * 0.02
    itm_put_strike = round((atm_strike_price + strike_increment) / 100) * 100
    otm_put_strike = round((atm_strike_price - strike_increment) / 100) * 100

    print(f"ITM put strike: {itm_put_strike}")
    print(f"OTM put strike: {otm_put_strike}")

    # Separate put options for the ATM and OTM strikes
    put_options_atm = options_for_closest_expiry[(options_for_closest_expiry['Instrument Type'] == 'PE') & (options_for_closest_expiry['Strike Price'] == atm_strike_price)]
    put_options_otm = options_for_closest_expiry[(options_for_closest_expiry['Instrument Type'] == 'PE') & (options_for_closest_expiry['Strike Price'] == otm_put_strike)]
    put_options_itm = options_for_closest_expiry[(options_for_closest_expiry['Instrument Type'] == 'PE') & (options_for_closest_expiry['Strike Price'] == itm_put_strike)]

    if put_options_atm.empty:
        print(f"ATM strike price of {atm_strike_price} on {selected_date} at {entry_time} not FOUND.")
        return pd.DataFrame()
    
    if put_options_otm.empty:
        print(f"OTM strike price of {otm_put_strike} on {selected_date} at {entry_time} not FOUND. Finding next closest strike price.")
        put_options_otm = options_for_closest_expiry[(options_for_closest_expiry['Strike Price'] > otm_put_strike) & (options_for_closest_expiry['Strike Price'] < atm_strike_price)]
        otm_put_strike = put_options_otm['Strike Price'].iloc[0]
        print(f"The next closest strike price is: {otm_put_strike}")

    if put_options_itm.empty:
        print(f"ITM strike price of {itm_put_strike} on {selected_date} at {entry_time} not FOUND. Finding next closest strike price.")
        put_options_itm = options_for_closest_expiry[(options_for_closest_expiry['Strike Price'] < itm_put_strike) & (options_for_closest_expiry['Strike Price'] > atm_strike_price)]
        itm_put_strike = put_options_itm['Strike Price'].iloc[-1]
        print(f"The next closest strike price is: {itm_put_strike}")

    atm_entry_price = put_options_atm.iloc[0]['Close']
    itm_entry_price = put_options_itm.iloc[-1]['Close']
    otm_entry_price = put_options_otm.iloc[0]['Close']
    total_entry_premium = (2 * atm_entry_price - itm_entry_price - otm_entry_price)
    print(f"Entry premium for one lot: {total_entry_premium}")
    total_entry_premium_lot = total_entry_premium * 15 * 1
    print(f"Total entry premium for 1 lots: {total_entry_premium_lot}")

    max_profit = (2 * atm_entry_price - otm_entry_price + (itm_entry_price - (itm_put_strike - atm_strike_price))) * 15 * 1
    print(f"Max profit in Rs.: {max_profit}")

    max_loss = min(
        (2 * atm_entry_price - otm_entry_price - itm_entry_price) * 15,
        (-otm_entry_price - (atm_strike_price - otm_put_strike) * 2 + (itm_put_strike - atm_strike_price)) * 15
    ) * 1
    print(f"Max loss in Rs.: {max_loss}")

    stop_loss = max_loss * 0.5
    target = max_profit * 0.7
    print(f"Stop loss: {stop_loss}")
    print(f"Target: {target}")

    trade_log = []

    # Log the trades
    trade_log = put_log_trades(trade_log, selected_date, closest_expiry, entry_time, atm_strike_price, put_options_atm.iloc[0], put_options_otm.iloc[0], put_options_itm.iloc[-1], capital, stop_loss, total_entry_premium_lot, target, max_profit, max_loss)

    # Update the global trade log
    trade_log_df = pd.concat([trade_log_df, pd.DataFrame(trade_log)], ignore_index=True)
    print(f"Trade log df is: {trade_log_df}")

    return trade_log_df

def automate_trading(pe_options_df_2018, spot_df_2018, initial_capital):
    unique_days = pe_options_df_2018['Datetime'].dt.date.unique()
    
    results_df = pd.DataFrame(columns=[
        'Serial_No', 'Date', 'Expiry', 'En_Date', 'Pe_En_Date', 'Ce_Ex_Date', 'Pe_Ex_Date',
        'Atm_Strike', 'Pe_Short_Strike', 'Pe_otm_Long_Strike', 'Pe_itm_Long_Strike', 'Capital',
        'Pe_Short_En_Price', 'Pe_Short_Ex_Price', 'Pe_otm_Long_En_Price', 'Pe_otm_Long_Ex_Price',
        'Pe_itm_Long_En_Price', 'Pe_itm_Long_Ex_Price', 'Entry_Time', 'Exit Time', 'Current Market Price',
        'Ticker_Pe_Short', 'Ticker_otm_Pe_Long', 'Ticker_itm_Pe_Long', 'Exit_Reason', 'Stop_Loss',
        'Total_Premium_Received', 'Target', 'Total_PnL', 'Max_Profit', 'Max_Loss'
    ])

    for day in unique_days:
        print("Processing trades for:", day)
        trade_data = create_straddle(pe_options_df_2018, spot_df_2018, str(day), '09:30:00', initial_capital)
        if not trade_data.empty:
            results_df = pd.concat([results_df, trade_data], ignore_index=True)
            monitor_trades(trade_data, pe_options_df_2018, pd.to_datetime(f"{day} 09:30:00"))
    
    return results_df

# Example call to the automate trading function
final_results_df = automate_trading(pe_options_df_2018, spot_df_2018, 1000000)
print(final_results_df)
